Installing required libraries for stepfunctions

In [1]:
 import sys
 !{sys.executable} -m pip install --upgrade pip
 !{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
 !{sys.executable} -m pip install -qU "stepfunctions>=2.0.0"
 !{sys.executable} -m pip show sagemaker stepfunctions

     |████████████████████████████████| 1.6 MB 22.7 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.1.2
    Uninstalling pip-21.1.2:
      Successfully uninstalled pip-21.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.21.13 which is incompatible.
Name: sagemaker
Version: 2.51.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages
Requires: attrs, boto3, pathos, numpy, packaging, smdebug-rulesconfig, protobuf3-to-dict, pandas, importlib-metadata, google-pasta, protobuf
Required-by: stepfunctions
---
Name: stepfunc

# 1. Setting up notebook with parameters and libraries

In [2]:
# importing Python libraies
import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from sagemaker.processing import Processor,ProcessingInput, ProcessingOutput
import calendar
import time
import sagemaker
from sagemaker.inputs import TrainingInput
import boto3
from sagemaker.network import NetworkConfig

stepfunctions.set_stream_logger(level=logging.INFO)

### Defining paramters

This needs to be changed if we are taking it to different environment

In [3]:
v_workflow_execution_role = "arn:aws:iam::525102048888:role/poc-sagemaker-step-functi-MachineLearningWorkflowE-1XFI2UPRXFTXE" # Step function IAM role ARN
v_preprocessing_iam_role = "arn:aws:iam::525102048888:role/service-role/AmazonSageMaker-ExecutionRole-20191105T125227" # IAM role for preprocessing container
v_preprocessing_instance_type = "ml.m5.xlarge" # Instance type for preprocessing container it changes as per workload
v_s3_input_bucket = "wipcoe-ml-s3-data-bucket" # S3 bucket for input and output data
v_prefix_for_input_data = "data/output/lr/baselineinp/train_baseline.csv"  # Prefix where data is stored
v_region = 'us-east-1' # AWS region
sec_groups = ["sg-044e0e7ce4f5721c0"]
subnets = ["subnet-0cf0e3f46326aa259",
           "subnet-0156b7f5500cf0b78",
           "subnet-032420199163cff9b"]
config_bucket = "wipcoe-datalake-init-s3-mlops-config"

## 2. Defining preprocessing jobs

In [4]:
# Defining environment config for baseline jobs
environment = {
          "dataset_format": "{\"csv\":{\"header\":\"True\"}}",
          "dataset_source": "/opt/ml/processing/input/baseline_dataset_input",
          "output_path": "/opt/ml/processing/output",
          "publish_cloudwatch_metrics": "Disabled"
        }

In [5]:
# Here we are creating baseline preprocesor
baseline_processor = Processor(image_uri='156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type,
                     network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets),
                     env=environment)

In [6]:
input_data = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_input_data)
inputs = [
    ProcessingInput(
        source=input_data, destination="/opt/ml/processing/input/baseline_dataset_input", input_name="input_data"
    )
]

outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/output",
        destination="s3://{}/{}".format(config_bucket,"customonitor/"),
        output_name="tr_data",
    )
]
gmt = time.gmtime()
ts = calendar.timegm(gmt)
baseline_name = "baseline-{}".format(ts)

In [7]:
print("s3://{}/{}".format(config_bucket,"customonitor"))


s3://wipcoe-datalake-init-s3-mlops-config/customonitor


In [8]:
baseline_preprocessing_step = steps.ProcessingStep(
    state_id='Baseline', 
    processor=baseline_processor,
    job_name=baseline_name, 
    inputs=inputs, 
    outputs=outputs, 
    experiment_config=None, 
    wait_for_completion=True
)

## 3. Step Function

In [9]:
# First we chain the start pass state,preprocessing_step,
basic_path=Chain([baseline_preprocessing_step])

In [10]:
# Next, we define the workflow
basic_workflow = Workflow(
    name="ds-mlops-dev-baseline-job",
    definition=basic_path,
    role=v_workflow_execution_role
)

#Render the workflow
basic_workflow.render_graph()

## 3.1 Create the workflow on AWS Step Functions

Create the workflow in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [11]:
basic_workflow.create()

[ERROR] A workflow with the same name already exists on AWS Step Functions. To update a workflow, use Workflow.update().


'arn:aws:states:us-east-1:525102048888:stateMachine:ds-mlops-dev-baseline-job'

In [12]:
basic_workflow.update(definition=basic_workflow.definition,role=basic_workflow.role)

[INFO] Workflow updated successfully on AWS Step Functions. All execute() calls will use the updated definition and role within a few seconds. 


'arn:aws:states:us-east-1:525102048888:stateMachine:ds-mlops-dev-baseline-job'

In [13]:
basic_workflow_execution = basic_workflow.execute(
    inputs={
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


## 3.2 Review the execution progress

Render workflow progress with the [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

This generates a snapshot of the current state of your workflow as it executes. This is a static image. Run the cell again to check progress. 

In [14]:
basic_workflow_execution.render_progress()

## 4 Downloading generated report on notebook

In [53]:
# Downloading contraint file for evaluation
!aws s3 cp s3://$config_bucket/monitoring/constraints.json .

download: s3://wipro-datalake-init-s3-mlops-config/monitoring/constraints.json to ./constraints.json


In [54]:
# Dowloading stats file for evaluation
!aws s3 cp s3://$config_bucket/monitoring/statistics.json .

download: s3://wipro-datalake-init-s3-mlops-config/monitoring/statistics.json to ./statistics.json


We can perform analysis on this file and put it on git hub

**Note :**
In order to refer it for modified location in monitoring schedule lambda function we need to changes the baseline stats location to prefix custom_monitoring/ from monitoring
